# Carregando bibliotecas

Bibbliotecas básicas

In [1]:
import numpy as np
import os
import sys
import pandas as pd
import unicodedata
import re
import collections

Bibliotecas para análises de redes

In [2]:
import igraph as ig
import networkx as nx

Bibliotecas para plots

In [3]:
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib as mpl
import seaborn as sns
%matplotlib notebook

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

#nuvem de palavras
import stylecloud as sc
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


Bad key text.latex.unicode in file C:\Users\teedo\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key text.latex.preview in file C:\Users\teedo\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 125 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file C:\Users\teedo\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 157 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably ne

# Importando os dados da tabela "INCT_IQ_data.xlsx"

In [4]:
def dec(name):
    aux_n = unicodedata.normalize('NFD', name)
    aux_n = aux_n.encode('ascii', 'ignore')
    aux_n = aux_n.decode("utf-8")
    return aux_n

bad = ['.','-']
def lev(v):
    v = v.replace('-',' ')
    v = v.replace('.','')
    v = v.replace(',','')
    v = v.upper()
    v1 = ' '.join(v.split())
    return v1

arq = "INCT_IQ_data.xlsx"

#variável com os dados dos pesquisadores: cidade, centro e artigos
df = pd.read_excel(arq, sheet_name="dados gerais pesquisadores A-Z", 
                   usecols=[0, 1, 2, 3, 4], names=['author', 'uf', 'city', 'center', 'papers'])
#limpeza da tabela df
df = df.drop(0)
df = df.drop(122)

keys = ['author', 'uf', 'city', 'center']
for key in keys:
    df[key] = df[key].map(str)
keys = ['papers']
for key in keys:
    df[key] = df[key].astype(float)
    
#variável com os dados dos pesquisadores: títulos, revistas e ano
au_names = ['title', 'revista', 'year']
au_names.extend(['authors' + '%d' % k for k in list(np.arange(0, 7, 1))])
df0 = pd.read_excel(arq, sheet_name="Art.2016-2022 com revista e ano", 
                   usecols=list(np.arange(0, 10, 1)), names=au_names)

#limpeza da tabela df0
keys = list(df0.keys())
for key in keys:
    df0[key] = df0[key].map(str)
keys = ['year']
for key in keys:
    df0[key] = df0[key].astype(float)
    
#juntar os nomes dos mesmos autores
df0["authors"] = df0["authors0"] + '|' + df0["authors1"] + '|' + df0["authors2"] + '|' + df0["authors3"] + '|' + df0["authors4"] + '|' + df0["authors5"] + '|' + df0["authors6"]

#variável com a abreviação dos nomes dos pesquisadores
names = ['Nome', 'Abrev']
df2 = pd.read_excel(arq, sheet_name="Correspondencia dos nomes", usecols=[0, 1], names = names)

#transformando todas as informações em strings
keys = list(df2.keys())
for key in keys:
    df2[key] = df2[key].map(str) 

# Nuvem de palavras de todos os títulos

In [5]:
text_all = " ".join(review for review in df0.title)

In [6]:
count_word = WordCloud().process_text(text_all)

In [7]:
sum([count_word[k] for k in count_word]) #total de palavras

8474

Top $10$ palavras que mais se repetem

In [14]:
for k in sorted(count_word, key=count_word.get, reverse=True)[:10]:
    print(k, '\t', count_word[k])

quantum 	 300
entanglement 	 69
de 	 65
state 	 63
system 	 62
effect 	 44
Experimental 	 44
optical 	 43
using 	 42
dynamic 	 41


In [15]:
f = open('titles-papers.txt','w',encoding='utf-8')
f.write('%s\n' % text_all)
f.close()

In [19]:
stopwords = set(STOPWORDS)
stopwords.update(["de", "two", "using", "via", "para", "work", "s", "e", "da"])

#número máximo de palavras
max_words = 80
wc=sc.gen_stylecloud(
    file_path='titles-papers.txt', custom_stopwords=stopwords, icon_name = 'fas fa-square',
    max_words=max_words, size=800, max_font_size = 150, output_name='figuras/nuvem-all.png')


# Número de artigos publicados por ano

In [20]:
yearsl = [int(k) for k in list(df0.year)]
nyears = dict(collections.Counter(sorted(yearsl)))

In [21]:
nyears

{2016: 184, 2017: 169, 2018: 172, 2019: 185, 2020: 215, 2021: 175, 2022: 56}

# número de artigos publicados em revista específicas

In [24]:
revs = [k.lower() for k in list(df0.revista)] #lower porque tem a mesma revista escrita de formas diferentes
count_revs = dict(collections.Counter(sorted(revs)))

Top $10$ revistas

In [25]:
for k in sorted(count_revs, key=count_revs.get, reverse=True)[:10]: #ver todos os nomes
    print(k, '\t', count_revs[k])

physical review a 	 267
physical review letters 	 62
physical review b 	 44
quantum information processing 	 42
physical review e 	 35
scientific reports 	 26
physical review research 	 23
revista brasileira de ensino de física 	 22
optics letters 	 17
physics letters a 	 16


# Número de pesquisadores por artigo

In [9]:
n1 = [linha.replace("'","").split("|") for linha in df0.authors]
#remore nan from the list
n2 = []
for n in n1.copy():
    cleanedList = [x for x in n if str(x) != 'nan']
    n2.append(cleanedList)
n1 = n2.copy()

n_au = dict(collections.Counter(sorted([len(k) for k in n1])))
n_au

{1: 892, 2: 166, 3: 64, 4: 15, 5: 16, 6: 2, 7: 1}

# Número de artigos publicados pelos pesquisadores

In [30]:
#Lista de pesquisadoras
woman1 = ['Aline Medina dos Santos', 'Bárbara Lopes Amaral', 'Belita Koiller', 
            'Ginette Jalbert de Castro Faria', 'Halyne Silva Borges', 'Liliana Sanz de la Torre', 
           'Malena Osório Hor-Meyll', 'Maria Carolina de Oliveira Aguiar', 'Nadja Kolb Bernardes', 
           'Paula Borges Monteiro', 'Thereza Cristina de Lacerda Paiva']

woman = ['MEDINA ALINE', 'AMARAL BARBARA', 'KOILLER BELITA', 'JALBERT GINETTE', 'AGUIAR M C O', 'BERNARDES N K',
        'MONTEIRO P B', 'PAIVA T']

In [34]:
prod_autor = {i:int(j) for i,j in zip(list(df.author), list(df.papers))}
number_aut = dict(collections.Counter(sorted(prod_autor.values())))

#número de artigos publicados por mulheres
womanp = sum([prod_autor[k] for k in prod_autor if k in woman1])
#número de artigos publicados por homens
manp = sum([prod_autor[k] for k in prod_autor]) - womanp

In [36]:
print(number_aut)

{0: 1, 1: 5, 2: 3, 3: 4, 4: 8, 5: 3, 6: 5, 7: 8, 8: 7, 9: 10, 10: 8, 11: 5, 12: 9, 13: 4, 14: 3, 15: 2, 16: 1, 17: 5, 18: 3, 19: 3, 20: 3, 21: 4, 22: 3, 23: 2, 24: 2, 26: 1, 29: 1, 32: 2, 37: 1, 38: 1, 47: 1, 49: 2, 57: 1}


# Número de pesquisadores, e publicações, em cada cidade

In [46]:
#1. dict com o nome do pesquisador e sua cidade
dcity = {i:j for i,j in zip(list(df.author), list(df.city))}
#2. dict com o nome do pesquisador e centro que atua
dcenter = {i:j for i,j in zip(list(df.author), list(df.center))}
#3. dict com o nome do pesquisador e quantidade de artigos
dpapers = {i:j for i,j in zip(list(df.author), list(df.papers))}

#número de pesquisadores por cidade
fcity = dict(collections.Counter(sorted(dcity.values())))
#número de artigos por cidade

#número de pesquisadores por centro
fcenter = dict(collections.Counter(sorted(dcenter.values())))

In [57]:
#número total de publicacções em cada cidade
pub_total = {}
for k in list(fcity.keys()):
    pub_total[k] = 0
for k in list(dpapers.keys()):
    if k in list(dcity.keys()):
        pub_total[dcity[k]] = pub_total[dcity[k]] + dpapers[k]
print(pub_total)

{'Arapiraca': 11.0, 'Araras': 7.0, 'Baurú': 13.0, 'Belo Horizonte': 141.0, 'Campinas': 115.0, 'Cornélio Procópio': 9.0, 'Cuiabá': 20.0, 'Curitiba': 23.0, 'Florianópolis': 74.0, 'Goiânia': 101.0, 'Itapeva': 7.0, 'Juiz de Fora': 4.0, 'Maceió': 21.0, 'Natal': 49.0, 'Niteroi': 161.0, 'Patrocínio': 6.0, 'Petrópolis': 4.0, 'Ponta Grossa': 26.0, 'Recife': 50.0, 'Rio de Janeiro': 293.0, 'Santa Maria': 26.0, 'Santo André': 47.0, 'Sorocaba': 2.0, 'São Carlos': 127.0, 'São Paulo': 116.0, 'Uberlândia': 11.0, 'Vitória': 57.0, 'Viçosa': 3.0, 'Volta Redonda': 57.0}


In [56]:
#número total de publicacções dos centros/universidades
pub_total_center = {}
for k in list(fcenter.keys()):
    pub_total_center[k] = 0
for k in list(dpapers.keys()):
    if k in list(dcenter.keys()):
        pub_total_center[dcenter[k]] = pub_total_center[dcenter[k]] + dpapers[k]
print(pub_total_center)

{'CBPF': 94.0, 'CEFET-RJ': 4.0, 'IFRJ - Volta Redonda': 20.0, 'IFSC': 1.0, 'IFTM': 6.0, 'IIF': 49.0, 'UEPG': 26.0, 'UFABC': 47.0, 'UFAL': 32.0, 'UFES': 57.0, 'UFF': 198.0, 'UFG': 101.0, 'UFJF': 4.0, 'UFMG': 141.0, 'UFMT': 20.0, 'UFPE': 44.0, 'UFPR': 23.0, 'UFRJ': 156.0, 'UFRJ/CCMN': 43.0, 'UFRPE': 6.0, 'UFSC': 73.0, 'UFSCar': 47.0, 'UFSCar/CCA': 7.0, 'UFSCar/CCTS': 2.0, 'UFSM/CCNE': 26.0, 'UFU': 11.0, 'UFV': 3.0, 'UNESP': 10.0, 'UNESP - Bauru': 13.0, 'UNESP-Itapeva': 7.0, 'UNICAMP': 115.0, 'USP': 116.0, 'USP/IFSC': 70.0, 'UTFPR': 9.0}


# Criação da rede de colaboração dos pesquisadores do INCT-IQ

In [10]:
#dict com as colaborações (relações) dos artigos
#######################################################################
############ CREDITOS: SAMURAÍ GOMES DE AGUIAR BRITO ##################
#######################################################################
net1 = {}
for l in range(len(n1)):
    aux = n1[l]
    if len(aux) == 1: #Caso em que não ocorre a colaboração em um artigo
        for i in aux:
            i = dec(i)
            i = lev(i)
            i = i.lstrip()
            if re.sub('[.-]','', i).replace(' ','').isalpha() and len(i.split())>1:
                net1.setdefault(i.strip('71!/#&*$?+$'), {})
    else:
        for i in aux:
            i = dec(i)
            i = lev(i)
            i = i.lstrip()
            for j in aux:
                j = dec(j)
                j = lev(j)
                j = j.lstrip()
                if i!=j and re.sub('[.-]','', i).replace(' ','').isalpha() and re.sub('[.-]','', j).replace(' ','').isalpha():
                    net1.setdefault(i.strip('71!/#&*$?+$'),{})[j.strip('71!/#&*$?+$')] = 0
                    net1.setdefault(j.strip('71!/#&*$?+$'),{})[i.strip('71!/#&*$?+$')] = 0

In [11]:
v_names_all = sorted(list(dict.fromkeys(net1).keys()))
print('Número total de artigos:                            %d' % len(df0))
print('Número total de pesquisadores:                      %d' % len(v_names_all))

Número total de artigos:                            1156
Número total de pesquisadores:                      120


In [12]:
#######################################################################
############ CREDITOS: SAMURAÍ GOMES DE AGUIAR BRITO ##################
#######################################################################

def network(net):
    
    list_name = sorted(list(net.keys()))
    vnum = {v:i for i,v in enumerate(list_name)}
    numv = {i:v for i,v in enumerate(list_name)}
    
    N = len(list_name)

    edges = [(vnum[k], vnum[val]) for k in list(net.keys()) for val in net[k]]
    G = ig.Graph(N, directed = False)
    G.add_edges(edges)
    G.vs["label"] = list_name
    G.simplify()
    
    return G

In [13]:
g = network(net1)
print('vértices: ', g.vcount(), '\nArestas: ', g.ecount())

vértices:  120 
Arestas:  282


Propriedades estatísticas da rede

In [14]:
degree_sequence = dict(collections.Counter(g.degree()))

gig = g.clusters().giant()
avg_dist = gig.average_path_length(directed=False)
size_gig = gig.vcount()
diameter = gig.diameter(directed=False)
assort = gig.assortativity_degree()
coef_agg = sum([k for k in g.transitivity_local_undirected() if k > 0])/g.vcount()

Calculo das comunidades

In [15]:
g.write_graphml('graph_colaboracao_INCT.graphML')

In [ ]:
#networkx
gnx = nx.read_graphml('graph_colaboracao_INCT.graphML')

#maior aglomerado
Gcc = sorted(nx.connected_components(gnx), key=len, reverse=True)
gtnx = gnx.subgraph(Gcc[0])
communit = nx_comm.louvain_communities(gtnx, resolution=1, seed=None)

print('Número de comunidades: ', len(communit))
print('\n')
print(communit)

# Rede da interação entre as cidades

In [73]:
#Dados geograficos para vizualizar a rede no Gephi no layout "Map of Countries" e #Geo Layout#
latitude = {}
longitude = {}

latitude['Arapiraca'], longitude['Arapiraca'] = -9.75164, -36.6604
latitude['Araras'], longitude['Araras'] = -22.3577, -47.3849
latitude['Baurú'], longitude['Baurú'] = -22.3154, -49.0615
latitude['Belo Horizonte'], longitude['Belo Horizonte'] = -19.8157, -43.9542
latitude['Campinas'], longitude['Campinas'] = -22.9064, -47.0616
latitude['Cornélio Procópio'], longitude['Cornélio Procópio'] = -23.1813, -50.648
latitude['Cuiabá'], longitude['Cuiabá'] = -15.5989, -56.0949
latitude['Curitiba'], longitude['Curitiba'] = -25.4284, -49.2733
latitude['Florianópolis'], longitude['Florianópolis'] = -27.5969, -48.5495
latitude['Goiânia'], longitude['Goiânia'] = -16.6799, -49.255
latitude['Itapeva'], longitude['Itapeva'] = -23.9676, -48.901
latitude['Juiz de Fora'], longitude['Juiz de Fora'] = -21.7642, -43.3496
latitude['Maceió'], longitude['Maceió'] = -9.66625, -35.7351
latitude['Natal'], longitude['Natal'] = -5.79448, -35.211
latitude['Niteroi'], longitude['Niteroi'] = -22.8808, -43.1043
latitude['Patrocínio'], longitude['Patrocínio'] = -18.9441, -46.9924
latitude['Petrópolis'], longitude['Petrópolis'] = -22.5046, -43.1823
latitude['Ponta Grossa'], longitude['Ponta Grossa'] = -25.0945, -50.1633
latitude['Recife'], longitude['Recife'] = -8.05428, -34.8813
latitude['Rio de Janeiro'], longitude['Rio de Janeiro'] = -22.9035, -43.2096
latitude['Santa Maria'], longitude['Santa Maria'] = -29.6914, -53.8008
latitude['Santo André'], longitude['Santo André'] = -23.6666, -46.5322
latitude['Sorocaba'], longitude['Sorocaba'] = -23.5062, -47.4559
latitude['São Carlos'], longitude['São Carlos'] = -22.0154, -47.8911
latitude['São Paulo'], longitude['São Paulo'] = -23.5489, -46.6388
latitude['Uberlândia'], longitude['Uberlândia'] = -18.9113, -48.2622
latitude['Vitória'], longitude['Vitória'] = -20.3222, -40.3381
latitude['Viçosa'], longitude['Viçosa'] = -20.7546, -42.8825
latitude['Volta Redonda'], longitude['Volta Redonda'] = -22.5252, -44.1038

In [88]:
abrev_names = {}
abrev_names_inv = {}
for key, name in zip(df2.Abrev, df2.Nome):
    abrev_names[key] = name
    abrev_names_inv[name] = key

In [103]:
city_au = {}
for key in list(abrev_names_inv.keys()):
    city_au[abrev_names_inv[key]] = dcity[key]

In [111]:
def geo_graph(net, city_au, latitude, longitude):
    list_name = sorted(set(list(fcity.keys())))
    vnum = {v:i for i,v in enumerate(list_name)}
    numv = {i:v for i,v in enumerate(list_name)}
    N = len(list_name)
    
    lt = [latitude[k] for k in list_name]
    lg = [longitude[k] for k in list_name]

    edges = [(vnum[city_au[k]], vnum[city_au[val]]) for k in list(net.keys()) for val in net[k] if val<k]
    G = ig.Graph(N, directed = False)
    G.add_edges(edges)
    G.vs["label"] = list_name
    G.vs["city"] = list_name
    G.vs["lat"] = lt
    G.vs["lng"] = lg
    
    return G

In [112]:
g_cities = geo_graph(net1, city_au, latitude, longitude)
#Vizualização no Gephi com permisão de auto-loops e Estrategia de mesclagem para arestas = Soma

In [113]:
print('vértices: ', g_cities.vcount(), '\nArestas: ', g_cities.ecount())

vértices:  29 
Arestas:  282


In [114]:
g_cities.write_graphml('graph_colaboracao_cidades_INCT.graphML')